<a href="https://colab.research.google.com/github/opedoussaut/orbit/blob/release%2Fv0.2.0/ORBIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORBIT
## Structuring circular life-cycle knowledge with ontologies  
### Version: v0.2.0 (in progress)
> Semantic contract framework for LCA, metrics, circularity, and EPD-ready reporting.


## ORBIT v0.2.0 (Contract-based validation)

This version introduces semantic contracts for:
- LCA structural integrity
- LCIA metrics comparability
- Circularity assumptions
- Reporting and EPD readiness

Contracts are expressed as SHACL shapes and applied to RDF graphs
instantiated from LCA study data.

In [ ]:
# Paths to ORBIT contract packs
CONTRACT_PATHS = {
    "lca_core": "contracts/lca-core.shacl.ttl",
    "lcia_metrics": "contracts/lcia-metrics.shacl.ttl",
    "circularity": "contracts/circularity.shacl.ttl",
    "reporting_epd": "contracts/reporting-epd.shacl.ttl",
}

# Paths to vocabularies
VOCAB_PATHS = {
    "metrics": "vocab/metrics.skos.ttl",
    "methods": "vocab/methods.skos.ttl",
    "boundaries": "vocab/boundaries.skos.ttl",
    "stages": "vocab/stages.skos.ttl",
}

# Paths to examples
EXAMPLE_PATHS = {
    "lca_core_ok":  "examples/lca-core_conforming.ttl",
    "lca_core_bad": "examples/lca-core_nonconforming.ttl",
}

In [ ]:
!pip -q install rdflib pyshacl

from rdflib import Graph
from pyshacl import validate

In [ ]:
import urllib.request

BRANCH = "release/v0.2.0"
BASE_RAW = f"https://raw.githubusercontent.com/opedoussaut/orbit/{BRANCH}"

def assert_url_ok(url: str):
    try:
        with urllib.request.urlopen(url) as r:
            return True
    except Exception as e:
        raise RuntimeError(f"Cannot load URL: {url}\n{e}")

def load_ttl_from_url(url: str) -> Graph:
    assert_url_ok(url)

    g = Graph()
    data = urllib.request.urlopen(url).read().decode("utf-8")
    g.parse(data=data, format="turtle")
    return g

shacl_g = load_ttl_from_url(f"{BASE_RAW}/{CONTRACT_PATHS['lca_core']}")

data_ok  = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lca_core_ok']}")
data_bad = load_ttl_from_url(f"{BASE_RAW}/{EXAMPLE_PATHS['lca_core_bad']}")

len(data_ok), len(data_bad), len(shacl_g)

In [ ]:
def run_contract(data_graph: Graph, shacl_graph: Graph, label: str):
    conforms, report_graph, report_text = validate(
        data_graph=data_graph,
        shacl_graph=shacl_graph,
        inference="rdfs",
        abort_on_first=False
    )
    print(f"\n=== {label} ===")
    print("CONFORMS:", conforms)
    print(report_text)

run_contract(data_ok, shacl_g, "LCA-CORE / Conforming example")
run_contract(data_bad, shacl_g, "LCA-CORE / Non-conforming example")